In [ ]:
# winequality-red.csv 파일의 데이터를 읽어서 테이블에 저장
# 1. 데이터베이스는 -2.ipynb에서 만든 data_repo 사용 (새로 만들기 X)
# 2. 테이블 만들기 (red_wine)
# 3. 데이터 읽어서 red_wine 테이블에 저장
# 4. 7등급 이상의 데이터를 조회해서 파일로 저장 (high-quality-red-wine.csv)

In [1]:
# 데이터베이스 연동 패키지(모듈) 준비
import pymysql

In [2]:
# 2. 테이블 만들기

try:
    # 데이터베이스 연결
    conn = pymysql.connect(
        host="127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    # 명령 실행기 준비
    cursor = conn.cursor()

    # 명령 실행
    sql = """create table if not exists redwine
          (
            idx int primary key auto_increment,
            fixed_acidity float not null,
            volatile_acidity float not null,
            citric_acid float not null,
            residual_sugar float not null,
            chlorides float not null,
            free_sulfur_dioxide float not null,
            total_sulfur_dioxide float not null,
            density float not null,
            pH float not null,
            sulphates float not null,
            alcohol float not null,
            quality int not null
          )"""
    cursor.execute(sql)    
except Exception as e:
    print("오류 발생 : ", e)
finally:
    # 연결 종료
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [3]:
# 데이터 분석 패키지 준비
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
# 3. 파일 데이터 읽어서 위에서 만든 테이블에 저장 

# 데이터 준비 ( 파일에서 데이터 읽기 )
df_iris = pd.read_csv("data-files/winequality-red.csv", sep=";")

try:
    # 데이터베이스 연결
    conn = pymysql.connect(
        host="127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    # 명령 실행기 준비
    cursor = conn.cursor()

    # 명령 실행
    sql = "delete from redwine"
    cursor.execute(sql)
    sql = """insert into redwine 
             (fixed_acidity, volatile_acidity, citric_acid, residual_sugar, chlorides, free_sulfur_dioxide, total_sulfur_dioxide, density, pH, sulphates, alcohol, quality)
             values 
             (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    data = df_iris.values.tolist()
    cursor.executemany(sql, data)

    conn.commit() # 마지막 commit or rollback 이후에 실행된 모든 변경사항 확정
except Exception as e:
    print("오류 발생 : ", e)
    if conn:
        conn.rollback() # 마지막 commit or rollback 이후에 실행된 모든 변경사항 취소
finally:
    # 연결 종료
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [8]:
# 4. 데이터베이스의 데이터 중 quality가 7 이상인 데이터를 무작위 조회해서 파일로 저장

try:
    # 데이터베이스 연결
    conn = pymysql.connect(
        host="127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    # 명령 실행기 준비
    cursor = conn.cursor()

    # 명령 실행
    sql = """select * from redwine where quality = %s""" # lower : 소문자로 변경
    cursor.execute(sql, [7])

    iris_data = cursor.fetchall() # 조회 결과를 tuple 형식으로 반환
    # print(type(iris_data))
    # print(iris_data[:3])

    # 파일로 저장
    df_iris = pd.DataFrame(iris_data, 
                           columns=['idx','fixed_acidity','volatile_acidity','citric_acid','residual_sugar','chlorides','free_sulfur_dioxide','total_sulfur_dioxide','density','pH','sulphates','alcohol','quality'])
    # df_iris.drop(['idx'], axis=1) # 'idx' 컬럼 제거, 원본 유지, 변경된 복사본 반환
    df_iris.drop(['idx'], axis=1, inplace=True) # 'idx' 컬럼 제거, 원본 변경
    df_iris.to_csv('data-files/high-quality-red-wine.csv', index=False) # index 저장하지 않도록 설정
    
except Exception as e:
    print("오류 발생 : ", e)

finally:
    # 연결 종료
    if cursor:
        cursor.close()
    if conn:
        conn.close()